# 전처리 성능 비교
* <font size=4>극단적 전처리 차이로 유사도 성능 비교 실험</font>

In [1]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

import re

In [6]:
import json

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# train data 전처리

In [10]:
# dev.json 파일 업로드

with open('/content/drive/MyDrive/sts-전처리/klue-sts-v1.1/klue-sts-v1.1_train.json', 'r') as f:

  json_data_1 = json.load(f)

  json.dumps(json_data_1, ensure_ascii=False)

with open('/content/drive/MyDrive/sts-전처리/klue-sts-v1.1/klue-sts-v1.1_train.json', 'r') as f:

  json_data_2 = json.load(f)

  json.dumps(json_data_2, ensure_ascii=False)

In [12]:
# 전처리 두가지 1. 영어, 숫자, 특수문자 전처리 2. 영어만 소문자 처리

# 1번 전처리(영어, 숫자, 특수문자 전처리)
def delete_1(json_data):
  for idx in range(len(json_data)):
    json_data[idx]['sentence1'] = re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', json_data[idx]['sentence1'])
    json_data[idx]['sentence2'] = re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', json_data[idx]['sentence2'])
  return json_data

# 2번 전처리(영어만 소문자 처리)
def delete_2(json_data):
  for idx in range(len(json_data)):
    json_data[idx]['sentence1'] = json_data[idx]['sentence1'].lower()
    json_data[idx]['sentence2'] = json_data[idx]['sentence2'].lower()
  return json_data


In [18]:
train_1 = delete_1(json_data_1)
train_2 = delete_2(json_data_2)

In [19]:
# 전처리 결과 확인

print(train_1[7162]['sentence1'])
print(train_2[7162]['sentence1'])

올해 예산 이·전용분 억 원 중 억 원은 코박스 퍼실러티  가입을 위한 선급금으로 이미 집행했다
올해 예산 이·전용분 1723억 원 중 850억 원은 코박스 퍼실러티(covax facility) 가입을 위한 선급금으로 이미 집행했다.


In [20]:
# 전처리 파일 json파일 저장

with open('/content/drive/MyDrive/sts-전처리/train_1.json','w') as f:
  json.dump(train_1, f, ensure_ascii=False)

with open('/content/drive/MyDrive/sts-전처리/train_2.json','w') as f:
  json.dump(train_2, f, ensure_ascii=False)

# Data train 시킨후 dev를 예측한 output 가져와 매트릭스 연산
<font size=4>team lostcow의 robert모델을 기준으로 함(robert-base)</font>

<font size=2>출처: https://github.com/l-yohai/KLUE</font>

* <font size=4>1. 영어 숫자, 특수문자 전처리 f1 스코어</font>

In [11]:
#Regression pred 를 binary-label pred로 변환

df_pred = pd.read_csv('/content/drive/MyDrive/sts-전처리/output_1.csv',header=None)
df_pred=(df_pred>2.5).astype(int)

In [12]:
#valid의 label 값을 추출
def read_json(file_path):
    with open(file_path) as f:
        return json.load(f)

valid_data=read_json('/content/drive/MyDrive/sts-전처리/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]

In [13]:
from sklearn.metrics import classification_report,f1_score

#혼돈 메트릭스 및 f1_score 시각화

print(classification_report(valid_label, df_pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, df_pred))

              precision    recall  f1-score   support

           0       1.00      0.65      0.79       299
           1       0.68      1.00      0.81       220

    accuracy                           0.80       519
   macro avg       0.84      0.83      0.80       519
weighted avg       0.86      0.80      0.80       519

base_line(roberta-base) f1_score: 0.8088235294117647


* <font size=4>2. 영어만 소문자 전처리 f1 스코어</font>

In [14]:
#Regression pred 를 binary-label pred로 변환

df_pred = pd.read_csv('/content/drive/MyDrive/sts-전처리/output_2.csv',header=None)
df_pred=(df_pred>2.5).astype(int)

In [16]:
#혼돈 메트릭스 및 f1_score 시각화

print(classification_report(valid_label, df_pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, df_pred))

              precision    recall  f1-score   support

           0       0.99      0.65      0.79       299
           1       0.68      0.99      0.80       220

    accuracy                           0.80       519
   macro avg       0.83      0.82      0.80       519
weighted avg       0.86      0.80      0.79       519

base_line(roberta-base) f1_score: 0.8044280442804427


### 결론
* <font size=4>전처리가 많은 데이터가 근소하게 높은 f1스코어를 보임</font>